<div>
<img src='https://upload.wikimedia.org/wikipedia/commons/6/64/Sharif_University_Logo.jpg' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center>
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homeworks 8 & 9 (Combined): Practical - Faces <br> <br>
<font color=686880 size=4>
    TAs: Arman Malekzadeh - Amirhossein Ramazani Bonab - Yalda Shabanzadeh
    
____

### Full Name : pardis zahraei
### Student Number : 99109777
### Colab Link: https://colab.research.google.com/drive/1j3AERkSioKgmnCNVbvBLXyS9HU4uI59-?usp=sharing
___

In this assignment, you'll be dealing with a dataset consisting of 400 faces, belonging to 40 people. Your main task is to train models capable of recognizing those faces.
The faces are `jpg` images packed together as a zip file called `images.zip`, and the `meta.csv` determines from which person each image was taken.

In [1]:
!wget -O images.zip https://www.dropbox.com/s/wrias0bjyte3rr2/images.zip?dl=1
!wget -O meta.csv https://www.dropbox.com/s/vqt1v8sfmz18rcf/faces-metadata.csv?dl=1

--2023-06-27 20:01:14--  https://www.dropbox.com/s/wrias0bjyte3rr2/images.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/wrias0bjyte3rr2/images.zip [following]
--2023-06-27 20:01:14--  https://www.dropbox.com/s/dl/wrias0bjyte3rr2/images.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbba2254dfd4c661174181928ec.dl.dropboxusercontent.com/cd/0/get/B-ynfxhPUDybHBqKWb8boENrZFkCQAzKE-kE1j0dlz7mvVTrbT2lDsMcQ6v6388ii_vLZv05-My1ElnxqxZ4GNZemET7O3LALzwByo-5ZtyOYGF5LeU88XMIdGzT8zPswSYmQAXqQ8fYXB1Bnacdmj00lf8U91I38chzxO0yBYlE7m67BQLwnIrT5U8Hshz0W0U/file?dl=1# [following]
--2023-06-27 20:01:14--  https://ucbba2254dfd4c661174181928ec.dl.dropboxusercontent.com/cd/0/get/B-ynfxhPUDybHBqKWb8boENrZFkCQAzKE-kE1j0dlz7mvVTrbT2lDsMcQ6v6388

In [2]:
# you can use the following packages
# The only added packages are zipfile for openning zip and cvxopt for solving QP and
# the Keras pacjage which was given initially was used for Neural network I used it for flipping,pooling, convoulotion
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
import plotly.express as px
import zipfile
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
import cvxopt
from cvxopt import matrix
from itertools import combinations
from sklearn.metrics import silhouette_score
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Sec 1: Data Preparation

Load all images as a numpy array (called `x`) and the metadata as a dataframe **(P1-1: 25 points)**

In [3]:
with zipfile.ZipFile("images.zip", "r") as zip_ref:
    images = [np.array(Image.open(zip_ref.open(j))) for j in [f"content/img-data/{i}.jpg" for i in range(400)]]
    x = np.array(images)

Flatten and normalize the images **(P1-2: 10 points)**

Note: For this part, normalization only means dividing by 255.

In [4]:
x = x.reshape(x.shape[0], -1)
x = x.astype(float) / 255.0

Encode the labels using `LabelEncoder` and save the result as a numpy array called `y` **(P1-3: 5 points)**

In [5]:
metadata = pd.read_csv('meta.csv')
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(metadata["label"].values)

Split the data into training (80%), validation (10%), and testing (10%) parts. **(P1-4: 10 points)**

Note: The split procedure should maintain the class balance. This is sometimes called "[stratified sampling](https://en.wikipedia.org/wiki/Stratified_sampling)".

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, stratify=y_test, random_state=42)

# Sec 2: SVM

## Subsec 2.1.

* I didnt know if its wanted to use gridsearch on validation set or on training set then give the accuracy so I did all the combinations possible.

Train a SVM using `sklearn`. Note that the hyperparameter tuning should be done using the "validation" set. **(P2-1-1: 20 points)**

In [7]:
# probably the wanted way which is grid search on validation set
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4],
    'gamma': ['scale', 'auto']
}
grid_search = GridSearchCV(SVC(), param_grid, cv=2)
grid_search.fit(x_val, y_val)
svm = SVC(**grid_search.best_params_)
svm.fit(x_train, y_train)
y_test_pred = svm.predict(x_test)
accuracy = accuracy_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred, average='weighted')
print("Best Parameters:", grid_search.best_params_)
print(f'Accuracy test: {accuracy:.4f}')
print(f'F1-score test: {f1:.4f}')

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(


Best Parameters: {'C': 0.1, 'degree': 2, 'gamma': 'scale', 'kernel': 'linear'}
Accuracy test: 0.9750
F1-score test: 0.9667


In [8]:
# the other way using train for CV
svm_model = SVC()
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4],
    'gamma': ['scale', 'auto']
}
grid_search = GridSearchCV(svm_model, param_grid, scoring='accuracy', cv=5)
grid_search.fit(x_train, y_train)
best_hyperparameters = grid_search.best_params_
best_model = grid_search.best_estimator_

y_val_pred = best_model.predict(x_val)
validation_accuracy = accuracy_score(y_val, y_val_pred)
print(f'Accuracy validation: {validation_accuracy:.4f}')

x_train_val = np.concatenate((x_train, x_val), axis=0)
y_train_val = np.concatenate((y_train, y_val), axis=0)
final_model = SVC(**best_hyperparameters)
final_model.fit(x_train_val, y_train_val)
y_test_pred = final_model.predict(x_test)
testing_accuracy = accuracy_score(y_test, y_test_pred)
print(f'Accuracy test: {testing_accuracy:.4f}')

Accuracy validation: 0.9750
Accuracy test: 0.9750


Report the accuracy and F1-score of the classifier on the testing set **(P2-1-2: 10 points)**

In [9]:
print(f'Accuracy test: {accuracy:.4f}')
print(f'F1-score test: {f1:.4f}')

Accuracy test: 0.9750
F1-score test: 0.9667


## Subsec 2.2.

Train a SVM from scratch using the following class **(P2-2-1: 40 points)**

Note: For this part, you are not required to use the validation set for hyperparameter tuning. However, you might get better results if you do that.

In [10]:
# one-vs-one approach
# I used OVO because dataset was quite small and it could be trained on it
# first it calculates the binary SVM using cvxopt to solve QP problem for it and then it checkes all combinations
# using voting scheme
class SVM_own:
    def __init__(self):
        self.classifiers = []

    def fit(self, X, y):
        self.classes = np.unique(y)
        for i, j in combinations(self.classes, 2):
            X_train = X[(y == i) | (y == j)]
            y_train = y[(y == i) | (y == j)]
            y_train = np.where(y_train == i, -1, 1)
            n_samples, n_features = X_train.shape
            P = matrix(np.outer(y_train, y_train) * np.dot(X_train, X_train.T))
            q = matrix(-np.ones(n_samples))
            G = matrix(-np.eye(n_samples))
            h = matrix(np.zeros(n_samples))
            A = matrix(y_train, (1, n_samples), 'd')
            b = matrix(0.0)
            sol = cvxopt.solvers.qp(P, q, G, h, A, b)
            alphas = np.array(sol['x']).flatten()
            sv_idx = np.where(alphas > 1e-5)[0]
            alphas = alphas[sv_idx]
            w = np.sum(alphas * y_train[sv_idx] * X_train[sv_idx].T, axis=1)
            b = np.mean(y_train[sv_idx] - np.dot(X_train[sv_idx], w))
            self.classifiers.append((i, j, w, b))

    def transform(self, X):
        votes = np.zeros((X.shape[0], len(self.classes)))
        for i, j, w, b in self.classifiers:
            pred = np.sign(np.dot(X, w) + b)
            pred = np.where(pred == -1, i, j)
            for k in range(X.shape[0]):
                votes[k][pred[k]] += 1
        y_pred = np.argmax(votes, axis=1)
        return y_pred


Report the accuracy and F1-score of the classifier on the testing set **(P2-2-2: 10 points)**

In [11]:
svm = SVM_own()
svm.fit(x_train, y_train)
y_pred = svm.transform(x_test)
test_accuracy = accuracy_score(y_test, y_pred)
test_f1_score = f1_score(y_test, y_pred, average="weighted")
print("Test accuracy:", test_accuracy)
print("Test F1-score:", test_f1_score)

Streaming output truncated to the last 5000 lines.
 6: -3.5643e-02 -3.5644e-02  1e-06  7e-18  3e-15
 7: -3.5644e-02 -3.5644e-02  1e-08  5e-18  4e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.8018e-02 -8.3513e-02  2e+01  4e+00  1e+00
 1: -3.1052e-03 -1.0738e-01  4e-01  7e-02  2e-02
 2: -1.6936e-02 -6.5682e-02  5e-02  1e-17  1e-14
 3: -2.6178e-02 -3.0479e-02  4e-03  5e-18  7e-15
 4: -2.6822e-02 -2.7188e-02  4e-04  7e-18  4e-15
 5: -2.6989e-02 -2.7004e-02  2e-05  3e-18  3e-15
 6: -2.6999e-02 -2.6999e-02  2e-07  7e-18  4e-15
 7: -2.6999e-02 -2.6999e-02  2e-09  7e-18  3e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.0460e-02 -6.0183e-02  2e+01  4e+00  1e+00
 1: -4.3371e-03 -7.4925e-02  6e-01  1e-01  3e-02
 2: -6.8180e-03 -5.2143e-02  5e-02  2e-17  6e-15
 3: -1.6669e-02 -2.1258e-02  5e-03  3e-18  2e-15
 4: -1.7449e-02 -1.8015e-02  6e-04  2e-18  2e-15
 5: -1.7659e-02 -1.7701e-02  4e-05  4e-18  2e-15
 6: -1.7682e-02 -1.76

# Sec 3: PCA

Fetch all images taken from `jones`, `taylor`, `anderson` and `wilson` (use the `labelencoder` object). Perform PCA from scratch using the following `class` to reduce the dimension of those images to 3. Save the results as a `numpy` array of shape `(40, 3)`. **(P3-1: 30 points)**

In [12]:
class PCA:
    def __init__(self, n_components):
        self.n_components = n_components
        self.components_ = None
        self.mean_ = None

    def fit(self, X):
        self.mean_ = np.mean(X, axis=0)
        X = X - self.mean_
        cov = np.cov(X.T)
        e_val, e_vec = np.linalg.eigh(cov)
        idx = e_val.argsort()[::-1]
        e_val = e_val[idx]
        e_vec = e_vec[:, idx]
        self.components_ = e_vec[:, :self.n_components]

    def transform(self, X):
        X = X - self.mean_
        return np.dot(X, self.components_)

names = ['jones', 'taylor', 'anderson', 'wilson']
idx = np.isin(metadata['label'], names)
X = x[idx]
Y = y[idx]
pca = PCA(n_components=3)
pca.fit(X)
X_pca = pca.transform(X)
np.save('X_pca.npy', X_pca)

Use `plotly` to plot these 3D samples. You should use their labels as the determiner of their color. **(P3-2: 10 points)**


In [13]:
fig = px.scatter_3d(
    x=X_pca[:, 0],
    y=X_pca[:, 1],
    z=X_pca[:, 2],
    color=label_encoder.inverse_transform(Y),
)
fig.show()


# Sec 4: K-Means

Using K-means, we want to perform clustering on the 3D samples. Consider 4 clusters. After running the K-means algorithm on the data, determine to which cluster each sample belongs. Then, use `plotly` to plot the 3D samples colored with respect to their clusters. **(P4-1: 15 points)**

In [14]:
class KMeans:
    def __init__(self):
        self.centroids = None

    def fit(self, X):
        n_samples, n_features = X.shape
        self.centroids = X[np.random.choice(n_samples, 4, replace=False)]
        for i in range(400):
            labels = np.argmin(np.linalg.norm(X[:, np.newaxis] - self.centroids, axis=2), axis=1)
            new_centroids = np.array([X[labels == k].mean(axis=0) for k in range(4)])
            if np.allclose(self.centroids, new_centroids):
                break
            self.centroids = new_centroids

    def predict(self, X):
        return np.argmin(np.linalg.norm(X[:, np.newaxis] - self.centroids, axis=2), axis=1)

kmeans = KMeans()
kmeans.fit(X_pca)
y_clus = kmeans.predict(X_pca)

fig = px.scatter_3d(
    x=X_pca[:, 0],
    y=X_pca[:, 1],
    z=X_pca[:, 2],
    color=y_clus,
)
fig.show()


Measure the quality of the clustering using "silhoutte score" **(P4-2: 5 points)**

Note: You can use `sklearn` or other packages to calculate the score.

In [15]:
score = silhouette_score(X_pca, y_clus)
print("Silhouette Score : {:.3f}".format(score))

Silhouette Score : 0.376


Compare the result of the clustering with the original labels. To be more precise, in this part, you should clearly answer the following questions with details:<br>

- Does each cluster represent a true label?
- What is your conclusion?

**(P4-3: 10 points)**

In [16]:
# using contingency_table to find out
contingency_table = pd.crosstab(y_clus, label_encoder.inverse_transform(Y))
contingency_table

col_0,anderson,jones,taylor,wilson
row_0,,,,
0,0,5,0,1
1,0,0,10,1
2,5,3,0,4
3,5,2,0,4


it is not accurate but given the amount of data it was given I think it works pretty well.
for the first question it kinda does I mean looking at the PCA and the amount of data it really does what a KNN does but it doesnt work very precise. taylor is pretty dense so its obvious it will be in a single cluster and 1 or 3 data (depends on every time we run it) is next to wilson so its going to wilson and wilson data is pretty much across the plane so its no shock it doesnt work well on willson. In the end no clustering isnt a good method it doesnt seperate the datas perfectly but its much easier and more visual so it can be helpful but not advised for clustering. not every cluster represent a true label and it can have errors.
to improve our performance I think the option is to increase the number of data by rotating or changing the scale which will be done in next part :)

# Sec 5: Neural Net (Optional with Bonus Points)

Reshape the images so that they become of shape `64x64` **(P5-1: 5 points)**

In [17]:
x_reshaped = x.reshape(-1, 64, 64)
x_train, x_test, y_train, y_test = train_test_split(x_reshaped, y, test_size=0.2, stratify=y, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, stratify=y_test, random_state=42)

Use different types of augmentation techniques (such as Random Flipping, Random Rotation, etc.) to increase the size of the training dataset. **(P5-2: 10 points)**

In [18]:
new_data = ImageDataGenerator(
    horizontal_flip=True,
    rotation_range=30,
    width_shift_range=0.25,
    height_shift_range=0.25,
)
x_train = np.expand_dims(x_train, axis=-1)
new_data.fit(x_train)
x_augmented,y_augmented = [],[]
for x_batch, y_batch in new_data.flow(x_train, y_train, batch_size=32):
    x_augmented.append(x_batch)
    y_augmented.append(y_batch)
    if len(x_augmented) == 20:
        break

x_train_augmented = np.concatenate([x_train, np.concatenate(x_augmented)])
y_train_augmented = np.concatenate([y_train, np.concatenate(y_augmented)])

Using the augmented training set, train a neural network consisting of 2D convolutional layers. The exact architecture of the network is arbitrary. However, the last layer should have the "softmax" function as its activation. Also, using "max pooling" layers after convolutional ones is advised.

Note: Don't get frustrated if your network does not appear to be a good one. Give it some time to be trained. For instance, you might need to train it for 100 epochs. Make sure to try that out before quitting!

**(P5-3: 20 points)**

In [19]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(40, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(x_train_augmented, y_train_augmented,
                    epochs=120,
                    validation_data=(x_val, y_val))

Epoch 1/120
30/30 [==============================] - 10s 280ms/step - loss: 3.6931 - accuracy: 0.0260 - val_loss: 3.6817 - val_accuracy: 0.0250
Epoch 2/120
30/30 [==============================] - 14s 456ms/step - loss: 3.6846 - accuracy: 0.0260 - val_loss: 3.6777 - val_accuracy: 0.0250
Epoch 3/120
30/30 [==============================] - 10s 338ms/step - loss: 3.6809 - accuracy: 0.0510 - val_loss: 3.6734 - val_accuracy: 0.0500
Epoch 4/120
30/30 [==============================] - 6s 184ms/step - loss: 3.6788 - accuracy: 0.0469 - val_loss: 3.6671 - val_accuracy: 0.0500
Epoch 5/120
30/30 [==============================] - 7s 237ms/step - loss: 3.6603 - accuracy: 0.0500 - val_loss: 3.6131 - val_accuracy: 0.0750
Epoch 6/120
30/30 [==============================] - 6s 186ms/step - loss: 3.5798 - accuracy: 0.0625 - val_loss: 3.3813 - val_accuracy: 0.1750
Epoch 7/120
30/30 [==============================] - 7s 240ms/step - loss: 3.2352 - accuracy: 0.1646 - val_loss: 2.7600 - val_accuracy: 0.2

Measure the accuracy of the network on the validation and testing datasets **(P5-4: 5 points)**

In [20]:
val_loss, val_acc = model.evaluate(x_val, y_val)
test_loss, test_acc = model.evaluate(x_test, y_test)

print(f'Validation accuracy: {val_acc:.4f}')
print(f'Test accuracy: {test_acc:.4f}')

2/2 [==============================] - 0s 22ms/step - loss: 0.2354 - accuracy: 0.9000
Validation accuracy: 0.9000
Test accuracy: 0.9000


I used 120 epoch and 30 batches for better result but it will be the same in 60 epochs as well